In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_glycoscience import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

from dgl import save_graphs, load_graphs

from sklearn.metrics import r2_score

### Explaniation of 2D GlycoNMR code at https://github.com/Cyrus9721/GlycoNMR

##### 1, Create Node embeddings

<div>
<img src="Picture1.png" width="500"/>
</div>

##### Initialize atom-level node embeddings based on extracted atom and monosaccharide features. 

##### Due to the inconsistent of data formating, as GlycoscienceDB is retrieved from various labs, the method of creating node embedding is slighly different from GODESS.

1, atom_dim: dimension of the atom name that indicate ring position, Eg: 04, C4, C3...

2, mono_dim: dimension of the monosaccharide name, glc.

3, ab_dim: dimension of the anomeric orientation of hydroxyl group.

4, dl_dim: dimension of the fischer convention, fischer d, l.

5, pf_dim: dimension of the number of carbon p, f.

6, residual_dim: dimension of the residual number starting from the root as number 1.



In [2]:
atom_dim = 256

residual_dim = 128

mono_dim = 64
ab_dim = 64
dl_dim = 64
pf_dim = 64

seed=9721

In [3]:
C = create_node_embeddings(seed=seed)

df_atom_embedding, df_residual_embedding, df_monosaccharide_embedding, df_ab_embedding, df_dl_embedding, df_pf_embedding =\
C.create_all_embeddings(atom_dim=atom_dim, residual_dim=residual_dim, mono_dim=mono_dim, 
                       ab_dim=ab_dim, dl_dim=dl_dim, pf_dim=pf_dim)



100%|██████████| 299/299 [00:00<00:00, 661.29it/s]


In [4]:
df_atom_embedding.to_csv(C.out_atom_embed, index=False)
df_residual_embedding.to_csv(C.out_residual_embed, index=False)
df_monosaccharide_embedding.to_csv(C.out_monosaccharide_embed, index=False)

df_ab_embedding.to_csv(C.out_bound_ab, index = False) 
df_dl_embedding.to_csv(C.out_bound_dl, index = False) 
df_pf_embedding.to_csv(C.out_carbon_pf, index = False) 

##### 2, Create adjaency matrix

<div>
<img src="Picture2.png" width="500"/>
</div>

##### Build adjaency matrix based on threshold distance between atoms

1, threshold_carbon: we construct edges between two carbon atom if their distance is within 1.65Å. 

2, threshold_hydrogen: we construct edges between two hydrogen atom if their distance is within 1.18Å. 

3, threshold_general: for all other atoms, we construct edges between distance two atom is within 1.5Å. 

4, threshold_interaction: we construct interaction edges between atoms if their distance is with in 5Å. 


In [5]:
B = build_adjacency_matrix()

In [6]:
B.calculate_all_matrix()

100%|██████████| 299/299 [00:05<00:00, 57.73it/s] 


##### 3, Create glycan graph

<div>
<img src="Picture3.png" width="800"/>
</div>

In [7]:
# shift_all = g.ndata["shift_value"].numpy()

# test_mask = g.ndata['test_carbon_mask'].numpy()

# test_shift = shift_all[test_mask]

# var_y = test_shift.var()



# print(var_y ** (1/2))


# test_shift.min(), test_shift.max()

# 1 - (1.2 ** 2) / var_y

In [8]:
np.sqrt(430)

20.73644135332772

##### create glycan graph from prebuilt node embeddings and adjaency matrix, every glycan is considered as a graph and every atom is consided as a node

1, num_test: number of testing glycans, we randomly select 20% of the glycans as test set


In [9]:
num_test = 60 # 299 * 20% 

In [10]:
Create = create_graph(num_test=num_test, seed=seed)
g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|██████████| 299/299 [00:09<00:00, 30.75it/s]


##### 4, Train a GNN model

<div>
<img src="Picture4.png" width="800"/>
</div>

In [11]:
num_epoch = 1000
lr = 1e-2

In [12]:
in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim


hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)
features = g.ndata["feat"]
labels = g.ndata["shift_value"]
# masks = g.ndata['train_mask'], g.ndata['test_mask']


# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']


print(features.dtype)
print(labels.dtype)
# model = NMR_GCN(in_size=576, hid_size=[256, 128, 64, 32], out_size=1).to(device)
model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)
# model training

# NMR_prediction = NMR_prediction(results_dir='experimental_data_combined/graph_combined/experimental_results_all_hydrogen.csv',
#                                 model_dir='experimental_data_combined/graph_combined/Model_hydrogen_experiment_training.pt',
#                                num_epoch = num_epoch, 
#                                lr = lr)


NMR_prediction = NMR_prediction(results_dir='glycosciencedb/results/training_carbon.csv',
                                results_dir_test = 'glycosciencedb/results/testing_carbon.csv',
                                model_dir='glycosciencedb/results/Model_Godess_carbon.pt', 
                               num_epoch = num_epoch, 
                               lr = lr)


print("Training...")
NMR_prediction.train(g, features, labels, masks, model, report_r2=True)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

# acc = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, print_out=True)
acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))

torch.float32
torch.float32
Training...


  0%|          | 1/1000 [00:00<14:40,  1.13it/s]

Epoch 00000 | Loss 6230.1094 | train_RMSE 63.2244 | train_r2 -977.9870 | test_RMSE 63.1746 | test_r2 -693.4457 


  0%|          | 2/1000 [00:01<14:36,  1.14it/s]

Epoch 00001 | Loss 3999.1035 | train_RMSE 122.7903 | train_r2 -17.8722 | test_RMSE 113.4124 | test_r2 -10.2595 


...

100%|█████████▉| 999/1000 [15:44<00:00,  1.04it/s]

Epoch 00998 | Loss 21.4904 | train_RMSE 2.9466 | train_r2 0.9791 | test_RMSE 3.3707 | test_r2 0.9735 


100%|██████████| 1000/1000 [15:45<00:00,  1.06it/s]

Epoch 00999 | Loss 21.4197 | train_RMSE 2.9788 | train_r2 0.9786 | test_RMSE 3.3898 | test_r2 0.9731 
best loss: 20.662137985229492
Testing...


train RMSE 3.0125
test RMSE 3.3806
